In [1]:
from openpyxl import load_workbook
import json
from datetime import datetime
import re
from konlpy.tag import Okt
import kss
import fitz
import difflib
import os

In [2]:
# 엑셀 읽기

def extract_data_from_excel(file_path):
    # Load the workbook
    workbook = load_workbook(filename=file_path)
    # Set the first sheet as the active sheet
    sheet = workbook.active

    # Data storage dictionary
    data_dict = {}

    # Variables to store the current key and the start row of that key
    last_key = None
    last_key_row = None

    # Iterate through each row in the sheet
    for row in sheet.iter_rows(min_row=1, max_col=sheet.max_column, values_only=False):
        current_row = row[0].row  # Current row number
        key = row[0].value  # Use the first column as the key

        # If a new key starts, process the previous key's data
        if key and last_key is not None:
            # Range of data collection for the previous key: from last_key_row to current_row - 1
            data_dict[last_key] = [sheet.cell(row=r, column=c).value
                                   for r in range(last_key_row, current_row)
                                   for c in range(2, sheet.max_column + 1)
                                   if sheet.cell(row=r, column=c).value is not None]
            last_key_row = current_row  # Update the start row for the new key

        # Store the new key
        if key:
            last_key = key
            last_key_row = current_row

    # Handle the last key
    if last_key is not None and last_key_row is not None:
        data_dict[last_key] = [sheet.cell(row=r, column=c).value
                               for r in range(last_key_row, sheet.max_row + 1)
                               for c in range(2, sheet.max_column + 1)
                               if sheet.cell(row=r, column=c).value is not None]

    return data_dict

In [3]:
# Json 형태로 변환

def default_converter(o):
    """Convert non-serializable objects for JSON."""
    if isinstance(o, datetime):
        return o.isoformat()
    raise TypeError(f'Object of type {o.__class__.__name__} is not JSON serializable')

def convert_to_json(data_dict):
    """
    Convert a dictionary to a JSON string, handling non-serializable objects.
    
    Args:
    data_dict (dict): The dictionary to serialize.

    Returns:
    str: JSON formatted string.
    """
    try:
        json_data = json.dumps(data_dict, indent=4, ensure_ascii=False, default=default_converter)
        return json_data
    except TypeError as e:
        print(f"Error during conversion: {e}")

In [4]:
# 빈 리스트 제거

def clean_empty_lists(json_data):
    """
    Remove keys with empty list values from a JSON string.

    Args:
    json_data (str): JSON string containing the data.

    Returns:
    str: Modified JSON string with empty lists removed.
    """
    try:
        # Convert JSON string to a Python dictionary
        data_dict = json.loads(json_data)

        # Find keys with empty lists to remove
        keys_to_remove = [key for key, value in data_dict.items() if isinstance(value, list) and not value]

        # Remove the identified keys
        for key in keys_to_remove:
            del data_dict[key]

        # Convert the modified dictionary back to a JSON string
        cleaned_json_data = json.dumps(data_dict, indent=4, ensure_ascii=False)
        return cleaned_json_data
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [5]:
# 키와 값 정제

def clean_json_text(json_data):
    """
    Clean text within the JSON data by removing HTML tags (excluding bold), replacing newlines with spaces,
    and reducing multiple spaces to a single space.

    Args:
    json_data (str): JSON string to be cleaned.

    Returns:
    str: A JSON string with cleaned text.
    """

    def clean_text(text):
        # Remove HTML tags (except bold)
        text = re.sub(r"</?\w+[^>]*>", "", text)
        # Replace newline characters with a space
        text = text.replace('\n', ' ')
        # Reduce multiple spaces to a single space
        text = re.sub(r'\s{2,}', ' ', text)
        return text

    try:
        # Convert JSON string to dictionary
        data_dict = json.loads(json_data)

        # Clean each key and value in the dictionary
        cleaned_data = {}
        for key, value in data_dict.items():
            # Clean the key
            clean_key = clean_text(key)
            
            # Clean the value; check if it's a list of strings
            if isinstance(value, list):
                clean_value = [clean_text(item) if isinstance(item, str) else item for item in value]
            else:
                clean_value = clean_text(value) if isinstance(value, str) else value
            
            cleaned_data[clean_key] = clean_value

        # Convert the cleaned dictionary back to a JSON string
        cleaned_json_data = json.dumps(cleaned_data, indent=4, ensure_ascii=False)
        return cleaned_json_data
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [6]:
def process_json_data(json_data):
    """
    Process the given JSON string to modify its structure based on specific rules:
    - Extracts specific values like "환매방법", "모집ㆍ매출 총액", and "존속기간" into separate keys.
    - Removes strings containing "위험" if they are less than 10 characters long.

    Args:
    json_data (str): JSON string to be processed.

    Returns:
    str: A modified JSON string.
    """
    try:
        # Convert the JSON string to a dictionary
        json_data_dict = json.loads(json_data)

        # Modified data dictionary
        modified_data = {}

        # Process each key and its associated values
        for key, values in json_data_dict.items():
            modified_values = []  # Store new values for the current key
            i = 0  # Index tracker for iterating through values
            while i < len(values):
                # Check for "환매방법", "모집ㆍ매출 총액", and "존속기간, " and handle separately
                if values[i] in ["환매방법", "환매 방법"] and i+1 < len(values):
                    modified_data["환매방법"] = [values[i+1]]  # Assign the next value under "환매방법"
                    i += 2  # Skip "환매방법" and its,  value
                elif values[i] in ["모집(매출) 총액","모집(매출)", "모집 매출 총액", "모〮집 매출총액", "모집ㆍ매출 총액", "모집ㆍ 매출총액", "모집·매출 총액", "모집,매출 총액"] and i+1 < len(values):
                    modified_data["모집ㆍ매출 총액"] = [values[i+1]]  # Assign the next value under "모집ㆍ매출 총액"
                    i += 2  # Skip "모집ㆍ매출 총액" and its value
                elif values[i] in ["존속기간", "존속 기간"] and i+1 < len(values):
                    modified_data["존속기간"] = [values[i+1]]  # Assign the next value under "존속기간"
                    i += 2  # Skip "존속기간" and its value
                else:
                    value = values[i]
                    # Skip strings containing "위험" if they are shorter than 10 characters
                    if isinstance(value, str) and "위험" in value and len(value) < 10:
                        i += 1
                        continue
                    modified_values.append(value)
                    i += 1
            if modified_values:  # If there are modified values, add them to the modified data
                modified_data[key] = modified_values

        # Convert the modified dictionary back to JSON string
        modified_json_data = json.dumps(modified_data, indent=4, ensure_ascii=False)
        return modified_json_data

    except json.JSONDecodeError as e:
        return f"Error decoding JSON: {e}"
    except Exception as e:
        return f"An error occurred: {e}"

In [7]:
def process_json_and_clean_text(json_data):
    """
    Processes JSON data to remove non-sentences based on morphological analysis and reassigns values based on specific rules for the key "[요약정보]".

    Args:
    json_data (str): JSON string containing lists of strings under various keys.

    Returns:
    str: A modified JSON string with non-sentences removed.
    """
    # Initialize the morphological analyzer
    okt = Okt()

    def is_valid_sentence(text):
        """Check if the text contains verbs or adjectives, indicating it's a sentence."""
        if not isinstance(text, str) or not text.strip():
            return False
        pos = okt.pos(text, norm=True, stem=True)
        return any(tag in ['Verb', 'Adjective'] for word, tag in pos)

    try:
        # Load the JSON data into a dictionary
        json_data_dict = json.loads(json_data)

        # Dictionaries to store cleaned and removed items
        cleaned_data = {}
        removed_items = {}

        # Keys to ignore during processing
        ignore_keys = ["분류", "모집ㆍ매출 총액", "모집ㆍ매출총액", "존속기간", "효력발생일", "효력 발생일", "모집기간", "매입방법", "매입 방법", 
                       "매입 방 법", "환매방법", "환매 방법", "환매 수수료", "환매수수료"]

        # Special processing for keys containing "[요약정보]"
        summary_key_substring = "[요약정보]"
        summary_keys = [key for key in json_data_dict.keys() if summary_key_substring in key]

        for summary_key in summary_keys:
            summary_values = json_data_dict[summary_key]
            new_summary_data = {
                "투자목적 및 투자전략": [],
                "분류": [],
                "투자비용": []
            }
            current_key = "투자목적 및 투자전략"
            
            for value in summary_values:
                if value == "분류":
                    current_key = "분류"
                elif value == "투자비용":
                    current_key = "투자비용"
                else:
                    new_summary_data[current_key].append(value)
            
            # Add new summary data to cleaned_data
            cleaned_data.update(new_summary_data)

        # Remove the summary keys
        for summary_key in summary_keys:
            if summary_key in cleaned_data:
                del cleaned_data[summary_key]

        # Iterate through each key and process text values
        for key, values in json_data_dict.items():
            # Ignore specific keys or already processed summary key
            if key in ignore_keys or key in summary_keys:
                cleaned_data[key] = values
                continue

            cleaned_values = []
            removed_values = []

            # Evaluate each text item in the list associated with the key
            for value in values:
                if is_valid_sentence(value):
                    cleaned_values.append(value)
                else:
                    removed_values.append(value)

            # Store cleaned values if not empty
            if cleaned_values:
                cleaned_data[key] = cleaned_values
            # Optionally, store removed values to keep track
            if removed_values:
                removed_items[key] = removed_values

        # Convert cleaned data back to JSON
        return json.dumps(cleaned_data, indent=4, ensure_ascii=False)

    except json.JSONDecodeError as e:
        return f"Error decoding JSON: {e}"
    except Exception as e:
        return f"An error occurred: {e}"

In [8]:
# 각 키의 VALUE 값을 하나의 문자열로 결합

def join_list_values_in_json(json_data):
    """
    Joins list values in each key of a JSON string into a single string.

    Args:
    json_data (str): A JSON string with dictionary entries containing lists of strings.

    Returns:
    str: A modified JSON string where each list of strings is joined into a single string.
    """
    try:
        # Load JSON data into a Python dictionary
        json_data_dict = json.loads(json_data)

        # Concatenate all list items into a single string for each key
        for key in json_data_dict:
            if isinstance(json_data_dict[key], list):  # Ensure the value is a list before joining
                json_data_dict[key] = ' '.join(str(item) for item in json_data_dict[key])

        # Convert the modified dictionary back into a JSON string
        return json.dumps(json_data_dict, indent=4, ensure_ascii=False)

    except json.JSONDecodeError as e:
        return f"Error decoding JSON: {e}"
    except Exception as e:
        return f"An error occurred: {e}"

In [9]:
def reindex_json_keys(json_data):
    """
    Reindexes the keys of a JSON string based on a predefined mapping.

    Args:
    json_data (str): A JSON string.

    Returns:
    str: A JSON string with reindexed keys based on predefined mapping.
    """
    try:
        # Load JSON data into a Python dictionary
        json_data_dict = json.loads(json_data)

        # Initialize the reindexed dictionary
        reindexed_dict = {}

        # Reindex keys using the predefined mapping
        for key, value in json_data_dict.items():
            new_key = None
            for index, keywords in key_map.items():
                if any(keyword in key for keyword in keywords):
                    new_key = index
                    break

            if new_key is not None:
                reindexed_dict[str(new_key)] = value
            else:
                print(f"Warning: Key '{key}' not found in key_map. Skipping.")

        # Convert the reindexed dictionary back to a JSON string
        return json.dumps(reindexed_dict, indent=4, ensure_ascii=False)
    except json.JSONDecodeError as e:
        return f"Error decoding JSON: {e}"
    except Exception as e:
        return f"An error occurred: {e}"

# Example usage:
# json_data = '...'  # your JSON data string
# updated_json = reindex_json_keys(json_data)
# print(updated_json)

In [10]:
def transform_json_to_array(json_data):
    """
    Transforms a JSON string that represents a dictionary into a JSON string that represents
    an array of objects with specified structure.

    Args:
    json_data (str): A JSON string representing a dictionary.

    Returns:
    str: A JSON string representing an array of new objects.
    """
    try:
        # Load JSON data from string to dictionary
        json_dict = json.loads(json_data)

        # Create a new list where each item is a dictionary with additional keys
        new_json_array = []
        for key, value in json_dict.items():
            new_json_array.append({
                "id": key,
                "index": 0,
                "bold": 0,
                "weight": 0,
                "text": value
            })

        # Convert the list back into a JSON string
        return json.dumps(new_json_array, indent=4, ensure_ascii=False)
    except json.JSONDecodeError as e:
        return f"Error decoding JSON: {e}"
    except Exception as e:
        return f"An error occurred: {e}"

In [11]:
def split_json_array_to_sentences(json_data):
    """
    Splits the text from a JSON array string, where each item contains a text field, into sentences using KSS.
    Each sentence retains its original metadata and gets an updated index.

    Args:
    json_data (str): A JSON string formatted as an array, where each entry contains a text field ("text") and may contain multiple sentences.

    Returns:
    str: A JSON string formatted as an array, where each entry is a dictionary with the metadata from the original array entry and the sentence text itself.
    """
    # Convert the JSON string to a list of dictionaries
    json_array = json.loads(json_data)

    # Create a new list to hold the structured sentence data
    new_json_array = []

    # Iterate through each dictionary in the list
    for item in json_array:
        # Use KSS to split the 'text' field into sentences
        sentences = kss.split_sentences(item['text'])
        # Append each sentence with its metadata to the new array
        for idx, sentence in enumerate(sentences):
            new_json_array.append({
                "id": item['id'],
                "index": idx,
                "bold": item.get('bold', 0),
                "weight": item.get('weight', 0),
                "text": sentence
            })

    # Sort the new_json_array by 'id' after converting 'id' to an integer
    new_json_array_sorted = sorted(new_json_array, key=lambda x: int(x['id']))

    # Convert the list back into a JSON string
    new_json_data = json.dumps(new_json_array_sorted, indent=4, ensure_ascii=False)
    return new_json_data

In [12]:
# 가중치 부여

def update_weight_based_on_ids(json_data, ids_to_update):
    """
    Updates the 'weight' field to 1 for specific item IDs in a JSON string that represents a list of dictionaries.

    Args:
    json_data (str): A JSON string containing a list of dictionaries.
    ids_to_update (list): A list of string IDs for which the 'weight' should be set to 1.

    Returns:
    str: A JSON string with updated 'weight' values.
    """
    # Convert the JSON string to a Python list of dictionaries
    data_list = json.loads(json_data)

    # Update 'weight' to 1 for specified IDs
    for item in data_list:
        if item['id'] in ids_to_update:
            item['weight'] = 1

    # Convert the updated list back to a JSON string
    updated_json_data = json.dumps(data_list, indent=4, ensure_ascii=False)
    return updated_json_data


specific_ids = ['0', '1', '2', '3', '6', '7', '10']

In [13]:
def extract_bold_and_red_text(file_path):
    """Extracts text that is either bold or red from a PDF file."""
    def to_rgb(color):
        """Convert a color from integer format to an RGB tuple."""
        r = (color >> 16) & 0xFF
        g = (color >> 8) & 0xFF
        b = color & 0xFF
        return r, g, b

    bold = []
    doc = fitz.open(file_path)  # Open the document

    for page_number in range(doc.page_count):
        page = doc.load_page(page_number)  # Load each page
        blocks = page.get_text("dict")["blocks"]  # Get text blocks

        for block in blocks:
            if "lines" in block:
                for line in block["lines"]:
                    for span in line["spans"]:
                        # Check if text is bold by font flag or style
                        if 'Bold' in span['font'] or span['flags'] & 2:
                            bold.append(span['text'])
                        
                        # Check if text is red by color
                        r, g, b = to_rgb(span['color'])
                        if r > g and r > b:
                            bold.append(span['text'])

    doc.close()  # Close the document
    return bold

In [14]:
def find_and_update_boldness(data, bold_sentences):
    """Updates the 'bold' attribute in the JSON data based on similarity to provided bold sentences."""
    similarity_threshold = 0.4  # Threshold for similarity to consider text as bold
    # Load JSON data
    data = json.loads(data)
    # Iterate through each item in the data
    for item in data:
        item_text = item['text']
        # Check against each bold sentence
        for bold_sentence in bold_sentences:
            # Compute the similarity ratio
            sequence_matcher = difflib.SequenceMatcher(None, item_text, bold_sentence)
            if sequence_matcher.ratio() > similarity_threshold:
                item['bold'] = 1  # Mark as bold if similarity is above the threshold
                break  # Stop after the first match to avoid redundant updates
    data = json.dumps(data, indent=4, ensure_ascii=False)
    return data

In [15]:
def split_text_into_sentences(text):
    """
    Splits the input text into sentences based on the period (.) character.
    """
    sentences = text.split('.')
    # Remove any leading or trailing whitespace and filter out empty sentences
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    return sentences

In [16]:
def add_document_field(json_data, document_number):
    """
    Adds a 'document_n' field to the JSON data.

    Args:
    json_data (str): A JSON string formatted as an array.
    document_number (str): The document number to be added.

    Returns:
    str: A JSON string with the 'document_n' field added.
    """
    try:
        # Convert the JSON string to a list of dictionaries
        data = json.loads(json_data)
        
        # Insert the 'document_n' field at the beginning
        data.insert(0, {"document_n": document_number})
        
        # Convert the list back into a JSON string
        new_json_data = json.dumps(data, indent=4, ensure_ascii=False)
        return new_json_data
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return json_data

In [17]:
def process_all_files_in_folder(folder_path):
    """Process all Excel files in the specified folder and print warnings for missing keys."""
    for filename in os.listdir(folder_path):
        if filename.endswith(".xlsx"):
            file_path_xl = os.path.join(folder_path, filename)
            print(f"Processing file: {file_path_xl}")
            
            try:
                data = extract_data_from_excel(file_path_xl)
            except Exception as e:
                print(f"Error processing file {file_path_xl}: {e}")
                print(f"Skipping file: {file_path_xl}")
                continue
            
            data = convert_to_json(data)
            data = clean_empty_lists(data)
            data = clean_json_text(data)
            data = process_json_data(data)
            data = process_json_and_clean_text(data)
            data = join_list_values_in_json(data)
            data = reindex_json_keys(data)

In [18]:
def save_all_files_in_folder(folder_path, start_document_number):
    """
    Process all Excel and PDF files in the specified folder and save the results as JSON files in a 'json' folder.
    """
    json_folder = os.path.join(folder_path, 'json')
    os.makedirs(json_folder, exist_ok=True)
    
    document_number = start_document_number

    for filename in os.listdir(folder_path):
        name, ext = os.path.splitext(filename)
        
        if ext == ".xlsx":
            file_path_xl = os.path.join(folder_path, filename)
            file_path_pf = os.path.join(folder_path, f"{name}.pdf")
            savename = os.path.join(json_folder, f"{document_number}.0_{name}.json")

            print(f"Processing Excel file: {file_path_xl}")
            
            try:
                # Excel
                data = extract_data_from_excel(file_path_xl)
                data = convert_to_json(data)
                data = clean_empty_lists(data)
                data = clean_json_text(data)
                data = process_json_data(data)
                data = process_json_and_clean_text(data)
                data = join_list_values_in_json(data)
                data = reindex_json_keys(data)
                data = transform_json_to_array(data)
                data = split_json_array_to_sentences(data)
                data = update_weight_based_on_ids(data, specific_ids)
                
            except Exception as e:
                print(f"Error processing Excel file {file_path_xl}: {e}")
                continue

            if os.path.exists(file_path_pf):
                print(f"Processing PDF file: {file_path_pf}")
                try:
                    # Pdf
                    bold_texts = extract_bold_and_red_text(file_path_pf)
                    joined_text = ' '.join(bold_texts)
                    bold = split_text_into_sentences(joined_text)

                    # 붉은, 굵은 글씨 인덱싱
                    data = find_and_update_boldness(data, bold)
                except Exception as e:
                    print(f"Error processing PDF file {file_path_pf}: {e}")
                    continue

            try:
                # Write the JSON string to a file
                data = add_document_field(data, str(document_number))
                
                with open(savename, 'w', encoding='utf-8') as file:
                    file.write(data)
                print(f"Saved JSON file: {savename}")
                document_number += 1  # Increment document number only if the file is processed successfully
            except Exception as e:
                print(f"Error saving JSON file {savename}: {e}")

In [19]:
# 매핑
key_map = {
    0: ["투자 위험 등급 2등급 [높은 위험]","투자 위험 등급 3등급 [다소 높은 위험]","투자 위험 등급 5등급 [낮은 위험]","투자 위험 등급 3등급 [다소 높은 위험]","투자 위험 등급 5등급 [낮은 위험]","투자위험등급 6등급 (매우 낮은 위험)","투자 위험 등급 2등급 [높은위험]","투자위험등급 2등급 [다소높은위험]","투자 위험 등급 5등급 [낮은 위험]","투자 위험 등급","투자 위험 등급 : 5 등급 (낮은 위험)","투자위험등급 3등급 [다소","투자위험등급 : 3등급 [다소높은위험]","투자위험등급: 2등급[높은 위험]","투자위험등급","투자위험등급 : 4등급[보통 위험]","투자위험등급: 2등급[높은 위험]","투자 위험 등급 5 등급 [낮은 위험]","투자 위험 등급 3등급[다소 높은 위험 ]","투자 위험 등급 4등급 (보통 위험)","투자위험등급 : 6등급[매우 낮은 위험]","투자 위험 등급 4등급 (보통 위험)","투자 위험 등급 3등급 (다소 높은 위험)","투자 위험 등급 4등급[ 보,통 위험 ]""투자위험등급 : 3등급[다소 높은 위험]","투자위험등급 : 3등급[다소 높은 위험]","투자 위험 등급 3등급 (다소 높은 위험)","투자위험등급 : 3등급[다소 높은 위험]","투자 위험 등급 4등급(보통위험)","투자 위험 등급 3등급 (다소 높은 위험)","투자위험등급 : 3등급[다소 높은 위험]","투자위험등급 : 4등급[보통 위험]","투자 위험 등급 3등급(다소높은위험)","투자 위험 등급 3등급 (다소 높은 위험)","투자위험등급 : 2등급[높은 위험]","투자위험등급 : 3등급[다소 높은 위험]","투자위험등급 : 3등급[다소 높은 위험]","투자위험등급 : 3등급[다소높은위험]","투자 위험 등급 6등급 [매우 낮은 위험]","투자 위험 등급 3등급 (다소 높은 위험)","투자 위험 등급 4 등급(보통 위험)","투자 위험 등급 4등급 [보통 위험]","투자위험등급 : 2등급 [높은위험]","투자 위험 등급 4등급 [보통 위험]","투자 위험 등급 3등급 [다소 높은 위험]","투자 위험 등급 5등급(낮은 위험)","투자 위험 등급 3등급 [다소 높은 위험]","투자 위험 등급 5 등급(낮은 위험)","투자 위험 등급 4등급 [보통 위험]","투자 위험 등급 3등급(다소 높은 위험)","투자 위험 등급 2 등급 [높은 위험]","투자위험등급 : 5등급[낮은 위험]","투자위험등급 3등급 [다소 높은 위험]","투자 위험 등급 3등급(다소 높은 위험)","투자 위험 등급 2 등급 [높은 위험]","투자 위험 등급 4등급(보통 위험)","투자 위험 등급 4등급(보통 위험)","투자위험등급 6등급[매우 낮은 위험]","투자 위험 등급 2등급(높은위험)","투자 위험 등급 3등급 [다소 높은 위험]","투자 위험 등급 3등급 [다소 높은 위험]","투자 위험 등급 3 등급 [다소 높은 위험]","투자위험등급 : 4등급[보통 위험]","투자위험등급 :4등급[보통 위험]","투자 위험 등급 3등급(다소 높은 위험)","투자 위험 등급 3등급(다소 높은 위험)","투자 위험 등급 4 등급 [ 보통 위험]","투자 위험 등급 5등급(낮은 위험)","투자 위험 등급 5등급(낮은 위험)","투자 위험 등급 5등급(낮은위험)","투자위험등급 4등급[보통 위험]","투자 위험 등급 2 등급 [높은 위험]","투자 위험 등급 4 등급 [보통 위험]","투자 위험 등급 6등급 [매우 낮은 위험]","투자 위험 등급 6등급 [매우 낮은","투자위험등급 5등급[낮은 위험]","투자 위험 등급 5등급 (낮은 위험)","투자 위험 등급 2 등급(높은 위험","투자 위험 등급 2등급 (높은 위험)","투자 위험 등급 4등급 [보통 위험]", "투자위험등급 3등급[다소 높은 위험]", "투자위험등급 4등급 [보통 위험]", "투자위험등급 2등급 [높은 위험]", "투자위험등급 2등급[높은 위험]", "투자 위험 등급 5등급 [낮은 위험]", "투자위험등급 : 3 등급 [중간 위험]","투자위험등급: 5등급[낮은 위험]", "투자위험등급 1등급[매우 높은 위험]", "1", "투자 위험 등급 2등급(높은 위험)", "투자위험등급 : 2등급[높은 위험]"], # 그냥 맨 앞에 요약 부분
    1: ["투자 목적 및 투자 전략","및 전략","투자목 적 및 투자전 략","투자목적", "투자전략", "투자목적 및 투자전략"],
    2: ["분류"],
    3: ["투자비 용","투자비용","투자 비용"],
    4: ["이(연평균 수익률, 단 위 %)","수익률,","(연평균수익률, 단위:%)","(연평균","수익률)","추이","투자실적","추이 (연평균 수익률, 단위: %)","투자실 적추이 (연평균 수익률, 단 위 %)","투자실적추 이 (연평균 수익률, 단위: %)","투자실 적추이 (연평 균 수익 률, 단 위 %)","투자실적추 이(연평균 수익률, 단위 %)","투자실적추이 (연평균 수익률)","(연평균 수익률, %)","(연평균수익률)","투자실적추이","투자실적추이 (연평균 수익률, 단 위 %)","투자 실적 추이 (연평 균 수익 률, 단 위 %)","투자실적추이 (연평균 수익률, 단위: %)","투자실적추이 (연평균 수익률)","투자실적추 이(연평균 수익률, 단 위 %)","(연평균 수익률)","투자실 적추이 (연평균 수익률)","투자실적추이 (연평균 수익률) (단위, %)","투자실 적추이 (연평균 수익률, 단위 %)","투자실적 추이", "투자실적 추이 (연평균", "투자실적추🕔 (연평균수익률) (단위:%)", "투자실적추이 (연평균수익 률, 단위: %)", "투자실적추이 (연평균수익률)", "투자실적 추이 (연평균 수익률, 단위: %)", "투자실적 추이 (연평균 수익률)", "투자실적 추이 (연평균 수익률, 세전기준)", "투자실적추 이 (연평균 수익률)", "투자실적추이 (연평균수익률, 단위: %)"],
    5: ["운용전문","운용관련 자문업자","운용 전문인력","인력","운용 전문인력","운용 전문인력","운용 전문 인력","운용 전문인력","운용전문인 력","운용전 문인력","운용전문 인력", "운용전문인력"],
    6: ["유의사항","투자자","투자자 유의 사항","투자 자 유 의사 항","투자자 유 의사항","투자자유의사항","투자자 유의사항", "투자자 유의사 항"],
    7: ["주요","주요 투자 위 험","주요투자위 험","주요 투자위 험","주요투자위험","주요투자 위험","주요 투자 위험", "주요 투자위험", "주요투 자위험"],
    8: ["환매방법", "환매 방법"],
    9: ["매입방법", "매입 방법", "매입 방 법"],
    10: ["환매수 수료","환매 수수 료","환매수수 료","환매 수수료", "환매수수료"],
    11: ["기준 가","기준가","기준 가격,"],
    12: ["과세"],
    13: ["및 방법","전환 절차 및 방 법","전환절차", "전환절,차 및 방법", "전환절차 및방법", "전환절 차 및 방법"],
    14: ["모집(판매)기 간","모집(판 매) 기간""모집(판매)기 간", "모집기간","모집(판매) 기간"],
    15: ["효력발생일", "효력 발생일"],
    16: ["모집(매출) 총액","모집ㆍ매출 총액", "모집ㆍ매출총액"],
    17: ["존속기간"], 
    18: ["참조"],
    19: ["기구의","집합투자","집합투자 기구의","종류","집합투자 기구의 종 류","집합투자 기구의 종 류","집합투자 기구의 종 류""집합투기 구의 종 류","집합 투자 기구 의 종 류","집합투 자기구 의 종 류","집합투자기 구의 종류","집합투자기구 종류", "집합투자 기구의 종류","집합투자 기구 의 종류", "집합투자기구 의 종류", "집합투자기구의 종류", "집합투 자기구 의 종류"]
}

In [22]:
# 파일 이름 지정
filename = "data\키움_K55301BA7008_간이투자설명서_미래에셋글로벌솔루션증권자투자신탁(주식-재간접형)_(20240510)"

# 파일 경로
file_path_xl = f'{filename}.xlsx'
file_path_pf = f'{filename}.pdf'

# 파일 저장 이름
savename = f'{filename}.json'

#### 테스트1

In [31]:
# Excel
data = extract_data_from_excel(file_path_xl)
data = convert_to_json(data)
data = clean_empty_lists(data)
data = clean_json_text(data)
# data = process_json_data(data)
# data = process_json_and_clean_text(data)
# data = join_list_values_in_json(data)
# data = reindex_json_keys(data)
# data = transform_json_to_array(data)
# data = split_json_array_to_sentences(data)
# data = update_weight_based_on_ids(data, specific_ids)

In [32]:
print(data)

{
    "투자 위험 등급 2등급 [높은 위험]": [
        "하나자산운용(주)는 이 투자신탁의 실제 수익률 변동성을 감안하 여 2등급으로 분류하였습니다."
    ],
    "1": [
        2,
        3,
        4,
        5,
        6
    ],
    "매우 높은 위험": [
        "높은 위험",
        "다소 높은 위험",
        "보통 위험",
        "낮은 위험",
        "매우 낮은 위험",
        "집합투자증권은 「예금자보호법」에 따라 보호되지 않는 실적배당상품으로 투자원금의 손실이 발생할 수 있으므로 투자에 신중을 기하여 주시기 바 랍니다. 집합투자재산을 상장주식 등에 투자함으로써 투자증권의 가격변동, 이자 율 변동 등 기타 거시경제지표의 변화에 따른 위험에 노출됩니다. 집합투자기구의 핵심위험에 대해 투자에 신중을 기하시기 바랍니다."
    ],
    "[요약정보]": [
        "[투자목적] 이 투자신탁은 국내주식을 법 시행령 제94조제2항제4호에서 규정하는 주된 투자대상자산으로 하여 업종대표 주 및 시장테마주 등에 투자하여 벤치마크 대비 초과수익을 목표로 합니다. 잔여재산은 채권 및 어음에 투자 하여 안정적인 이자 수익을 추구하고자 합니다. 그러나 이 투자신탁의 투자목적이 반드시 달성된다는 보장은 없으며, 집합투자업자·판매회사·신탁업자 등 이 투자신탁과 관련된 어떠한 당사자도 수익자에 대해 투자원금 의 보장 또는 투자목적의 달성을 보장하지 아니합니다.",
        "[투자전략] 이 투자신탁은 국내주식을 법 시행령 제94조제2항제4호에서 규정하는 주된 투자대상자산으로 하여 주식시장 상승에 따른 자본소득을 추구하는 한편, 잔여 신탁재산으로 채권 및 유동성자산에 투자하여 안정적인 이자수 익을 추구하는 전략을 실행합니다."
    ],
    "투자목적 및 전략": [
        " 포트폴리오 전략에 주력하면서 시장대비 초과

### 테스트2

In [23]:
# Excel
data = extract_data_from_excel(file_path_xl)
data = convert_to_json(data)
data = clean_empty_lists(data)
data = clean_json_text(data)
data = process_json_data(data)
data = process_json_and_clean_text(data)
data = join_list_values_in_json(data)
data = reindex_json_keys(data)
data = transform_json_to_array(data)
data = split_json_array_to_sentences(data)
data = update_weight_based_on_ids(data, specific_ids)

# Pdf
bold_texts = extract_bold_and_red_text(file_path_pf)
joined_text = ' '.join(bold_texts)
bold = split_text_into_sentences(joined_text)

# 붉은, 굵은 글씨 인덱싱
data = find_and_update_boldness(data, bold)

print(data)

[
    {
        "id": "0",
        "index": 0,
        "bold": 0,
        "weight": 1,
        "text": "미래에셋자산운용㈜는 이 투자신탁의 실제수익률변동성을감안 하여3등급으로분류하 였 습니다."
    },
    {
        "id": "0",
        "index": 1,
        "bold": 1,
        "weight": 1,
        "text": "집합투자증권은「예금자 보호법」에따라보호 되지 않는실적 배당 상품이며,집합투자기구재산을주로글로벌에투자하는 모투자신탁에투자함 으로재간접투자위험,국가 위험,환율변동위험등이있 으므로투자 에신중을기하 여주시기 바랍니다."
    },
    {
        "id": "1",
        "index": 0,
        "bold": 0,
        "weight": 1,
        "text": "2. 투자전략 미래에셋글로벌솔루션증권모투자신탁(주식-재간접형)에 80% 이상으로 투자합니다."
    },
    {
        "id": "2",
        "index": 0,
        "bold": 0,
        "weight": 1,
        "text": "투자신탁, 증권(재간접형), 개방형(중도환매가능), 추가형(추가납입가능), 모자형, 종류형"
    },
    {
        "id": "3",
        "index": 0,
        "bold": 0,
        "weight": 1,
        "text": "투자자가 부담하는 수수료 및 총보수 (단위: %) 1,000만원 투자시 투자자가 부담하는 투자기간별 총비용 예시 (단위:천원) 수수료미징구-오프라인(C) 수수료미징구-온라인(C-e) 수수료미징구-오프라인-개 인연금(C-P) 수수료미징구-오프라인-퇴 직연금(C-P2) (주1) “1,000만원 투자시 투자자가 부담하는 투자기간별 총비용

### 키값 찾기

In [20]:
# Example usage
folder_path = 'data/키움'  # 폴더 경로를 설정하세요
process_all_files_in_folder(folder_path)

Processing file: data/키움\키움_K55223DJ7176_간이투자설명서_KB다이나믹TDF2040증권자(주식혼합-재간접형)_(20230831).xlsx
Processing file: data/키움\키움_K55223DJ7408_간이투자설명서_KB다이나믹TDF2050증권자(주식혼합-재간접형)_(20230831).xlsx
Processing file: data/키움\키움_K55223DT0104_간이투자설명서_KB한국인덱스50청년형소득공제증권자(채권혼합)_(20240517).xlsx
Processing file: data/키움\키움_K55223E28026_간이투자설명서_KB머니마켓액티브증권(채권)_(20230616).xlsx
Processing file: data/키움\키움_K55232BI7236_간이투자설명서_NH-Amundi고배당주증권자투자신탁[주식]_(20231229).xlsx
Processing file: data/키움\키움_K55232C99469_간이투자설명서_NH-Amundi 필승 코리아 증권투자신탁[주식]_(20231018).xlsx
Processing file: data/키움\키움_K55232D64446_간이투자설명서_NH-Amundi100년기업그린코리아증권투자신탁(주식)_(20240524).xlsx
Processing file: data/키움\키움_K55233B10417_간이투자설명서_칸서스튼튼채권탄탄공모주증권투자신탁1호(채권혼합)_(20240328).xlsx
Error processing file data/키움\키움_K55233B10417_간이투자설명서_칸서스튼튼채권탄탄공모주증권투자신탁1호(채권혼합)_(20240328).xlsx: invalid literal for int() with base 10: '0.500'
Skipping file: data/키움\키움_K55233B10417_간이투자설명서_칸서스튼튼채권탄탄공모주증권투자신탁1호(채권혼합)_(20240328).xlsx
Processing file: data/키움\키움_K55235B

Processing file: data/키움\키움_K55307BC6960_간이투자설명서_유리글로벌거래소연금저축증권자투자신탁[주식]_(20231024).xlsx
Processing file: data/키움\키움_K55307D05035_간이투자설명서_유리필라델피아반도체인덱스증권자투자신탁H[주식]_(20240321).xlsx
Processing file: data/키움\키움_K55307D05910_간이투자설명서_유리필라델피아반도체인덱스증권자투자신탁UH[주식]_(20240321).xlsx
Processing file: data/키움\키움_K55310DA9563_간이투자설명서_코레이트코스닥벤처플러스증권투자신탁[주식혼합-파생형]_(20231122).xlsx
Processing file: data/키움\키움_K55310DZ1231_간이투자설명서_코레이트초단기금리혼합자산투자신탁_(20240227).xlsx
Processing file: data/키움\키움_K55310E70692_R3_3. 간이투자설명서_코레이트셀렉트단기채증권투자신탁[채권]_240207(20240207).xlsx
Processing file: data/키움\키움_K55311B67767_간이투자설명서_플러스텐배거중소형주증권투자신탁1호(주식)_(20231221).xlsx
Processing file: data/키움\키움_K55363B10198_R3_간이투자설명서_트러스톤아시아장기성장주연금저축증권자투자신탁[주식](20240503).xlsx
Processing file: data/키움\키움_K55363C09438_간이투자설명서_트러스톤핀셋중소형증권자투자신탁[주식]_(20240126).xlsx
Processing file: data/키움\키움_K55363DD7867_간이투자설명서_트러스톤ESG지배구조레벨업증권자투자신탁[주식]_(20240208).xlsx
Processing file: data/키움\키움_K55363DI7839_간이투자설명서_트러스톤코스닥벤처공모주리츠증권투자신탁(주식혼합-파생형)_(20240517).xl

Processing file: data/키움\키움_KR5105636884_간이투자설명서_삼성코리아대표증권자투자신탁제1호(주식)_(20240208).xlsx
Processing file: data/키움\키움_KR5105671378_간이투자설명서_삼성아세안증권자투자신탁 제2호(주식)_(20240503).xlsx
Processing file: data/키움\키움_KR5105675288_간이투자설명서_삼성글로벌Water증권자투자신탁제1호(주식-재간접형)_(20240523).xlsx
Processing file: data/키움\키움_KR5105706539_간이투자설명서_삼성글로벌클린에너지증권자투자신탁제1호(주식-재간접형)_(20240214).xlsx
Processing file: data/키움\키움_KR5105730547_간이투자설명서_삼성기초소재강국코리아증권자투자신탁제1호(주식)_(20240227).xlsx
Processing file: data/키움\키움_KR5105774339_간이투자설명서_삼성이머징다이나믹증권자투자신탁제1호(주식)_(20240126).xlsx
Processing file: data/키움\키움_KR5105888360_간이투자설명서_삼성클래식연금증권전환형자투자신탁제1호(주식)_(20240109).xlsx
Processing file: data/키움\키움_KR5105888402_간이투자설명서_삼성클래식30연금증권전환형자투자신탁제1호(채권혼합)_(20240109).xlsx
Processing file: data/키움\키움_KR5105903383_간이투자설명서_삼성CHINA20본토증권자투자신탁제2호(주식)_(20240419).xlsx
Processing file: data/키움\키움_KR5105903391_간이투자설명서_삼성CHINA20본토증권자투자신탁제1호(주식)_(20240419).xlsx
Processing file: data/키움\키움_KR5105904548_간이투자설명서_삼성WTI원유특별자산투자신탁 제1호(WTI원유-파생형)_(20240503).

Processing file: data/키움\키움_KR5217200637_간이투자설명서_우리그로스증권투자신탁(주식)5호_(20240130).xlsx
Processing file: data/키움\키움_KR5219333469_간이투자설명서_신한베스트크레딧단기증권자투자신탁[채권]_(20231227).xlsx
Processing file: data/키움\키움_KR5219515123_간이투자설명서_신한얼리버드증권자투자신탁[주식]_(20231123).xlsx
Processing file: data/키움\키움_KR5219523986_간이투자설명서_신한아름다운SRI그린뉴딜증권자투자신탁제1호[주식]_(20240202).xlsx
Processing file: data/키움\키움_KR5219732827_간이투자설명서_신한해피라이프연금글로벌ETF증권전환형자투자신탁제1호[주식-재간접형]_(20230920).xlsx
Processing file: data/키움\키움_KR5219756164_간이투자설명서_신한골드증권투자신탁제1호[주식]_(20231227).xlsx
Processing file: data/키움\키움_KR5221325578_간이투자설명서_키움 프런티어 개인용 MMF 제1호[국공채]_(20230906).xlsx
Processing file: data/키움\키움_KR5221463353_간이투자설명서_키움코리아에이스증권자투자신탁제1호[주식]_(20231214).xlsx
Processing file: data/키움\키움_KR5223144340_간이투자설명서_KB스타코리아리버스인덱스증권(주식-파생형)_(20230906).xlsx
Processing file: data/키움\키움_KR5223281472_간이투자설명서_KB연금가치주증권전환형자(주식)_(20240315).xlsx
Processing file: data/키움\키움_KR5223697677_간이투자설명서_KB베트남포커스증권자(주식혼합)(H)_(20231101).xlsx
Processing file: data/키움\키움_KR52

Processing file: data/키움\키움_KR5301AR1435_간이투자설명서_미래에셋스마트롱숏70증권자투자신탁1호(주식)_(20230630).xlsx
Processing file: data/키움\키움_KR5301AS9825_간이투자설명서_미래에셋자산배분TDF2030증권자투자신탁(채권혼합-재간접형)_(20230825).xlsx
Processing file: data/키움\키움_KR5301AW5173_간이투자설명서_미래에셋차이나그로스증권자투자신탁1호(주식)_(20240126).xlsx
Processing file: data/키움\키움_KR5301AX4464_간이투자설명서_미래에셋스마트섹터배분증권자투자신탁1호(주식)_(20240510).xlsx
Processing file: data/키움\키움_KR5301AY3044_간이투자설명서_미래에셋연금글로벌그로스증권자투자신탁1호(주식)_(20240313).xlsx
Processing file: data/키움\키움_KR5301AY9678_간이투자설명서_미래에셋글로벌헬스케어증권자투자신탁1호(주식)_(20240510).xlsx
Processing file: data/키움\키움_KR5301AY9785_간이투자설명서_미래에셋연금글로벌헬스케어증권자투자신탁1호(주식)_(20240510).xlsx
Processing file: data/키움\키움_KR5301AZ1716_간이투자설명서_미래에셋연금중국본토증권자투자신탁1호(주식-재간접형)_(20240510).xlsx
Processing file: data/키움\키움_KR5303AS5953_간이투자설명서_마이다스미소중소형주증권자투자신탁(주식)_(20230630).xlsx
Processing file: data/키움\키움_KR5303AX6681_간이투자설명서_마이다스단기국공채공모주증권투자신탁제1호(채권혼합)_(20231228).xlsx
Processing file: data/키움\키움_KR5305AU0727_간이투자설명서_유경PSG좋은생각자산배분형증권투자신탁(주혼)_(20231101).

### 파일 처리 및 저장

In [23]:
# Example usage
folder_path = 'data/키움'  # 폴더 경로를 설정하세요
start_document_number = 1001
save_all_files_in_folder(folder_path, start_document_number)

Processing Excel file: data/키움\키움_KR5235480351_간이투자설명서_피델리티글로벌증권자투자신탁A(주식-재간접형)_(20240308).xlsx
Processing PDF file: data/키움\키움_KR5235480351_간이투자설명서_피델리티글로벌증권자투자신탁A(주식-재간접형)_(20240308).pdf
Saved JSON file: data/키움\json\1001.0_키움_KR5235480351_간이투자설명서_피델리티글로벌증권자투자신탁A(주식-재간접형)_(20240308).json
Processing Excel file: data/키움\키움_KR5235690595_간이투자설명서_피델리티재팬증권자투자신탁(주식-재간접형)_(20240308).xlsx
Processing PDF file: data/키움\키움_KR5235690595_간이투자설명서_피델리티재팬증권자투자신탁(주식-재간접형)_(20240308).pdf
Saved JSON file: data/키움\json\1002.0_키움_KR5235690595_간이투자설명서_피델리티재팬증권자투자신탁(주식-재간접형)_(20240308).json
Processing Excel file: data/키움\키움_KR5235690694_간이투자설명서_피델리티미국증권자투자신탁(주식-재간접형)_(20240308).xlsx
Processing PDF file: data/키움\키움_KR5235690694_간이투자설명서_피델리티미국증권자투자신탁(주식-재간접형)_(20240308).pdf
Saved JSON file: data/키움\json\1003.0_키움_KR5235690694_간이투자설명서_피델리티미국증권자투자신탁(주식-재간접형)_(20240308).json
Processing Excel file: data/키움\키움_KR5235690793_간이투자설명서_피델리티아시아증권자투자신탁(주식)_(20240308).xlsx
Processing PDF file: data/키움\키움_KR5235690793_간이투자

In [ ]:
# 저장

# Write the JSON string to a file
with open(savename, 'w', encoding='utf-8') as file:
    file.write(data)